In [1]:
# RNN网络搭建
# 数据集使用的是 mnist数据集

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

print("Packages imported")

mnist = input_data.read_data_sets("data/", one_hot=True)
trainimgs, trainlabels, testimgs, testlabels = \
    mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
ntrain, ntest, dim, n_classes = trainimgs.shape[0], testimgs.shape[0],trainimgs.shape[1],trainlabels.shape[1]

print("MNIST Loaded")

Packages imported
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST Loaded


In [2]:
diminput = 28  # 输入层维度
dimhidden = 128 # 隐藏层的权重维度为 28*128 输出的小大为 128*1
dimoutput = n_classes
nsteps = 28 # 把每个数据分成28个step依次传入构建的网络中

weights = {
    'hidden':tf.Variable(tf.random_normal([diminput, dimhidden])),
    'out':tf.Variable(tf.random_normal([dimhidden, dimoutput]))
}

biases = {
    'hidden':tf.Variable(tf.random_normal([dimhidden])),
    'out':tf.Variable(tf.random_normal([dimoutput]))
}

In [5]:
from tensorflow.contrib import rnn
# 一个RNN网络的构建
def _RNN(_X, _W, _b, _nsteps, _name):
    # 输入的数据的每个维度为：[batchsize, netstep, diminput]
    # 我们需要先把它转换成： [netstep, batchsize, diminput]
    _X = tf.transpose(_X,[1,0,2])
    # 然后转换数据的形状   这样可以共同训练 输入层到隐藏层之间的权重
    _X = tf.reshape(_X, [-1,diminput])
    
    # 输入层 => 隐藏层
    _H = tf.matmul(_X, _W['hidden']) + _b['hidden']
    
    _Hsplit = tf.split(_H, _nsteps,0 )
    
    with tf.variable_scope(_name ) as scope:
        scope.reuse_variables()
        
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(dimhidden, forget_bias=1.0)
        _LSTM_O, _LSTM_S = rnn.static_rnn(lstm_cell, _Hsplit, dtype=tf.float32)
    
    # 输出层
    _O = tf.matmul(_LSTM_O[-1], _W['out']) + _b['out']
    
    return {
        'X':_X, 'H':_H,'Hsplit':_Hsplit,
        'LSTM_O':_LSTM_O,'LSTM_S':_LSTM_S,'O':_O
    }
print("Network Ready")

Network Ready


In [6]:
# 用 TensorFlow 定义一些操作
learning_rate = 0.001
x      = tf.placeholder("float", [None, nsteps, diminput]) # 输入
y      = tf.placeholder("float", [None, dimoutput])  # 输出

myrnn  = _RNN(x, weights, biases, nsteps, 'basic')
pred   = myrnn['O']  # 得到预测值
cost   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))  # 损失函数
optm   = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) # Adam Optimizer
accr   = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1), tf.argmax(y,1)), tf.float32))
init   = tf.global_variables_initializer()
print ("Network Ready!")


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Network Ready!


In [9]:
training_epochs = 15
batch_size      = 16
display_step    = 1
sess = tf.Session()
sess.run(init)
print ("Start optimization")
for epoch in range(training_epochs):
    avg_cost = 0.
    #total_batch = int(mnist.train.num_examples/batch_size)
    total_batch = 100
    
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, nsteps, diminput))
        # Fit training using batch data
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict=feeds)/total_batch
    # Display logs per epoch step
    if epoch % display_step == 0: 
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)   
        print (" Training accuracy: %.3f" % (train_acc))
        testimgs = testimgs.reshape((ntest, nsteps, diminput))
        feeds = {x: testimgs, y: testlabels}
        test_acc = sess.run(accr, feed_dict=feeds)  # 这里原有的代码还增加了：istate: np.zeros((ntest, 2*dimhidden))
        print (" Test accuracy: %.3f" % (test_acc))
print ("Optimization Finished.")

Start optimization
Epoch: 000/015 cost: 1.752700942
 Training accuracy: 0.562
 Test accuracy: 0.447
Epoch: 001/015 cost: 1.334377567
 Training accuracy: 0.375
 Test accuracy: 0.516
Epoch: 002/015 cost: 1.171451274
 Training accuracy: 0.688
 Test accuracy: 0.522
Epoch: 003/015 cost: 1.025814539
 Training accuracy: 0.688
 Test accuracy: 0.609
Epoch: 004/015 cost: 0.958811806
 Training accuracy: 0.562
 Test accuracy: 0.598
Epoch: 005/015 cost: 0.880201100
 Training accuracy: 0.875
 Test accuracy: 0.659
Epoch: 006/015 cost: 0.790639217
 Training accuracy: 0.875
 Test accuracy: 0.684
Epoch: 007/015 cost: 0.758282970
 Training accuracy: 0.625
 Test accuracy: 0.706
Epoch: 008/015 cost: 0.690140132
 Training accuracy: 0.875
 Test accuracy: 0.716
Epoch: 009/015 cost: 0.658807516
 Training accuracy: 0.688
 Test accuracy: 0.697
Epoch: 010/015 cost: 0.634672035
 Training accuracy: 0.688
 Test accuracy: 0.698
Epoch: 011/015 cost: 0.582864018
 Training accuracy: 0.688
 Test accuracy: 0.727
Epoch: 01